### Reporting workflow overview

Anchors:
- `analog_image_generator.reporting.build_reports`

Use this notebook to demonstrate the features listed below. Fill in the upcoming sections (presets, generator runs, stacked packages, metrics, reporting, and debugging hooks) before the professor review.


### anchor-reporting-pipeline

`analog_image_generator.reporting.build_reports(metrics_rows, output_dir)` writes the CSV plus per-env PDFs and master PDF; reporting rows combine stats output with realization metadata (env, seed, QA flags, stacked packages).


#### Demo checklist

- [ ] Construct sample `metrics_rows`
- [ ] Run `build_reports` to generate CSV/PDF/PNG artifacts
- [ ] Display thumbnails or metadata from outputs
- [ ] Document output paths + upload instructions
- [ ] QA checklist referencing smoke test


### anchor-reporting-mosaics

Internal helpers `_generate_mosaics` + `_build_env_pdfs` apply `docs/PALETTES.md` legends, embed grayscale/facies mosaics, histograms, and QA tables into the ReportLab PDFs appended into master_report.pdf.


In [1]:
checklist = {
    "metrics_ready": True,
    "anchors_synced": True,
}
assert all(checklist.values())
checklist


{'metrics_ready': True, 'anchors_synced': True}